In [1]:
import gym

In [2]:
import numpy as np
from keras.layers import Input, Conv2D, Dense, MaxPooling2D , Flatten
from keras.models import Model, Sequential, load_model
from collections import deque
from keras.optimizers import Adam
import keras

Using TensorFlow backend.


In [3]:
env = gym.make('SpaceInvaders-v0')

In [10]:
env.reset()
env.render()

True

In [18]:
ac =env.action_space.sample()

In [19]:
o,r,d,i = env.step(ac)
dqn = DQN([210,160] , 6)
dqn.get_action(o)

ValueError: Input 0 is incompatible with layer conv2d_13: expected ndim=4, found ndim=3

<TimeLimit<AtariEnv<SpaceInvaders-v0>>>

In [9]:
for t in range(1000):
    env.render()
    ac =env.action_space.sample()
    o,r,d,i = env.step(ac)
    if d:
        env.reset()
env.close()

In [7]:
env.close()

In [11]:
class DQN:
    def __init__(self, input_shape, output_shape, discount=0.99, update_target_every=10, memory_size=2000):
        self.input_shape=input_shape
        self.output_shape=output_shape
        self.discount=discount
        self.update_target_every=update_target_every
        self.policy_net=self.create_model()
        self.memory=deque(maxlen=memory_size)
        self.target_counter=0 
    
    def create_model(self):
        model=Sequential()
        model.add(Conv2D(input_shape=self.input_shape, filters=128, kernel_size=(7,7), strides=(1,1), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(MaxPooling2D(pool_size=(3,3), padding="valid"))
        model.add(Conv2D(filters=128, kernel_size=(7,7), strides=(2,2), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(MaxPooling2D(pool_size=(2,2), padding="valid"))
        model.add(Conv2D(filters=128, kernel_size=(7,7), strides=(2,2), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(Flatten())
        model.add(Dense(512, activation="relu"))
        model.add(Dense(self.output_shape, activation="softmax"))
        adm=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        model.compile(loss="mse", optimizer=adm, metrics=["accuracy"])
        return model
    
    def get_action (self,state):
        action_prob= self.policy_net.predict(state)
        return action_prob
        
       
        